In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [4]:
# Load dataset
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")

In [5]:
# Drop non-beneficial columns
application_df = application_df.drop(columns=['EIN', 'NAME'])


In [6]:
# Replace rare APPLICATION_TYPE values with "Other"
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() < 500].index.tolist()
application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(application_types_to_replace, "Other")


In [7]:
# Replace rare CLASSIFICATION values with "Other"
classifications_to_replace = application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() < 1000].index.tolist()
application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(classifications_to_replace, "Other")


In [8]:
# One-hot encode categorical data
application_df = pd.get_dummies(application_df)

In [9]:
# Split features and target
X = application_df.drop(columns=['IS_SUCCESSFUL'])
y = application_df['IS_SUCCESSFUL']

In [10]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [11]:
# Normalize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [12]:
# Create model with optimization methods
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [13]:
# Model definition
nn = Sequential()

In [14]:
# Input & First hidden layer with dropout
nn.add(Dense(units=100, activation="relu", input_dim=X_train_scaled.shape[1]))
nn.add(Dropout(0.2))

c:\Users\Jakeu\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
# Second hidden layer
nn.add(Dense(units=60, activation="relu"))

In [16]:
# Third hidden layer
nn.add(Dense(units=30, activation="relu"))

In [17]:
# Output layer
nn.add(Dense(units=1, activation="sigmoid"))

In [18]:
# Compile model (Optimization 2: Learning rate tuning via Adam optimizer)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
nn.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [19]:
# Optimization 3: Early Stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [20]:
# Train the model
fit_model = nn.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=100,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7008 - loss: 0.5949 - val_accuracy: 0.7337 - val_loss: 0.5557
Epoch 2/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 1s 827us/step - accuracy: 0.7267 - loss: 0.5624 - val_accuracy: 0.7364 - val_loss: 0.5517
Epoch 3/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 1s 784us/step - accuracy: 0.7249 - loss: 0.5598 - val_accuracy: 0.7347 - val_loss: 0.5505
Epoch 4/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 1s 799us/step - accuracy: 0.7250 - loss: 0.5584 - val_accuracy: 0.7376 - val_loss: 0.5493
Epoch 5/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 1s 812us/step - accuracy: 0.7271 - loss: 0.5574 - val_accuracy: 0.7374 - val_loss: 0.5496
Epoch 6/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 1s 812us/step - accuracy: 0.7295 - loss: 0.5530 - val_accuracy: 0.7359 - val_loss: 0.5531
Epoch 7/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 1s 767us/step - accuracy: 0.7359 - loss: 0.5469 - val_accuracy: 0.7376 - val_loss: 0.5490
Epoch 8/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 1s 801us/step - accuracy: 0.7284 - loss: 0.5

In [21]:
# Save model
nn.save("AlphabetSoupCharity_Optimization.h5")